Fredholm integral equation of the second kind using mechanical quadrature method (Book 2, p. 38, ex. 4.2.2).   

# Common parts

In [1]:
from numpy import  array, sum, cosh, zeros
from numpy.linalg import solve

import pandas as pd 

We are given task below, but solution will be appropriate for all fredholm equations of the second kind:

$u(x)-0.6 \int_{0}^{1} \operatorname{ch}(x y) u(y) d y=x-0.6$

In [2]:
def H(x, y):
    return cosh(x * y)

def f(x):
    return x - 0.6

c = 0.6

a = 0
b = 1

Because of the fact that we will use mechanical quadrature method, we will need quadrature formula. In current task we will use middle rectangles rule.

In [3]:
def middle_rectangles(f, N, a=0, b=1):
    """Composite middle rectangles formula.
    
    Args:
        f: Function that we want to count integral from.
        N (int): Number of partitions.
        a, b (int, int): Bounds.
        
    Returns:
        result (float): Approximate integral value.
    """
    
    # Length of part split.
    h = (b - a) / N
    
    # Points.
    x = array([a + (h / 2) + (k - 1) * h for k in range(1, N + 1)])
    
    # Values in points.
    y = f(x)
    
    # Integral value.
    result = h * sum(y)

    return result

But in current task we only need points and coefficents from quadrature formula.

In [4]:
def middle_rectangles_points_coefficents(N, a=0, b=1):
    """Composite middle rectangles formula points and coefficents.
    
    Args:
        N (int): Number of partitions.
        a, b (int, int): Bounds.
        
    Returns:
        points (list<float>).
        coefficents (list<float>).
    """
    
    # Length of part split.
    h = (b - a) / N
    
    points = array([a + (h / 2) + (k - 1) * h 
                    for k in range(1, N + 1)])
    
    coefficents = [h] * N

    return points, coefficents

Also at some point we will need kroneker delta.

In [5]:
def kroneker_delta(x, y): 
    return 1 if x == y else 0

# Mechanical quadrature method 

In [6]:
def mechanical_quadrature(H, f, N, find_points_coefficents, x, c=1, a=0, b=1):
    """Solving fredholm integral equation of the second kind. 
    
    Using mechanical quadrature method.   
    
    Args:
        H (func(x, y)): Function under integral.
        f (func(x)): Free value function.
        c (float): Constant before integral.
        a, b (float): Integral bounds.
        N (int): Number of points in quadrature method.
        find_points_coefficents (func(N, a, b)): Function that returns
                                              points and coefficents from 
                                              appropriate quadrature 
                                              formula.
        x (float): Point to count result function in.
    
    Returns:
        result (float): value of desired function in point x.
    """
    
    points, coefficents = find_points_coefficents(N, a, b)
    
    # Solving Dz=g system.
    D = zeros(shape=(N,N))
    for row in range(N):
        for col in range(N):
            D[row][col] = kroneker_delta(row, col) - \
                          coefficents[col] * \
                          H(points[row], points[col])
    g = f(points)
    z = solve(D, g)
    
    summ = 0
    for i in range(N):
        summ += coefficents[i] * H(x, points[i]) * z[i]
    
    result = summ + f(x)
    
    return result

# Testing

In [7]:
# Starting number of splits.
n = 5

# Number of calculations.
# Number of splits grows twice
# as much on every iteration.
calcs = 7

In [8]:
# Generating number of splits.
splits = [2**i*n for i in range(calcs)]

# Generating first column of table.
x_s = list(map(lambda x: 'u^({})(x)'.format(x), splits))

# Generating values in point a.
a_s = [mechanical_quadrature(H,
                             f, 
                             N=n,
                             find_points_coefficents=middle_rectangles_points_coefficents,
                             x=a, 
                             c=c,
                             a=a,
                             b=b) for n in splits]

# Generating values in point (a+b)/2.
a_b_s = [mechanical_quadrature(H,
                             f, 
                             N=n,
                             find_points_coefficents=middle_rectangles_points_coefficents,
                             x=(a+b)/2, 
                             c=c,
                             a=a,
                             b=b) for n in splits]

# Generating values in point b.
b_s = [mechanical_quadrature(H,
                             f, 
                             N=n,
                             find_points_coefficents=middle_rectangles_points_coefficents,
                             x=b, 
                             c=c,
                             a=a,
                             b=b) for n in splits]

In [9]:
# Creating table.
data = pd.DataFrame(list(zip(x_s, a_s, a_b_s, b_s)), 
                    columns =['$x$', '$a$', '$(a + b)/2$', '$b$'])

data.style.hide_index()

$x$,$a$,$(a + b)/2$,$b$
u^(5)(x),0.860636,1.435083,2.170189
u^(10)(x),0.824088,1.397904,2.131292
u^(20)(x),0.815062,1.388722,2.121684
u^(40)(x),0.812813,1.386433,2.119288
u^(80)(x),0.812251,1.385861,2.118690
u^(160)(x),0.812110,1.385718,2.118541
u^(320)(x),0.812075,1.385682,2.118503
